In [10]:
import pandas as pd
import fastf1 as ff1
from datetime import timedelta

In [11]:
# Configurazione della sessione
anno = 2023
gara = 'Monaco'
sessione = 'R'  # Race

In [12]:
# Carica la sessione
session = ff1.get_session(anno, gara, sessione)
session.load()

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
req            INFO 	Data has been written to cache!
core           INFO 	Loading data for Monaco Grand Prix - Race [v3.4.5]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_m

In [26]:
# Funzione per convertire timedelta in formato MM:SS.mmm
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    minutes, seconds = divmod(total_seconds, 60)
    milliseconds = int(td.microseconds / 1000)
    return f"{minutes:02}:{seconds:02}.{milliseconds:03}"

# Ottieni i dati meteo
weather_data = session.weather_data

# Ottieni i dati del pilota (esempio: Verstappen)
driver = 'HAM'
driver_laps = session.laps.pick_drivers([driver])
driver_telemetry = driver_laps.get_car_data().add_distance()

# Ottieni informazioni sul circuito
circuito = session.event.Location  # Nome del circuito

# Nota: FastF1 non fornisce direttamente la lunghezza del circuito, quindi usiamo una mappatura manuale
lunghezza_circuito = {
    'Monaco': 3337,  # Lunghezza del circuito di Monaco in metri
    'Silverstone': 5891,  # Lunghezza del circuito di Silverstone in metri
    'Spa': 7004,
    'Baku': 6003,
    'Zandvoort': 4252,
    'Imola': 4930,
    'Jeddah': 6203,
    'Suzuka': 5807,
    'Interlagos': 4309,
    'Austin': 5513,
    'Sochi': 5848,
    'Bahrain': 5412,
    'Melbourne': 5303,
    'Shanghai': 5451,
    'Montreal': 4361,
    'Le Castellet': 5842,
    'Red Bull Ring': 4318,
    'Silverstone': 5891,
    'Hungaroring': 4381,
    'Hockenheim': 4574,
    'Monza': 5793,
    'Las Vegas': 7004,
    'Mexico City': 4304,
    'Adelaide': 3780,
    'Phoenix': 3770,
    'Estoril': 4415,
    'Magny-Cours': 4411,
    'Indianapolis': 4180,
    'Sepang': 5543,
    'Istanbul': 5338,
    'Valencia': 4005,
    'Yeongam': 5603,
    'Portimao': 4653,
    'Jerez': 4423,
    'Istanbul': 5338,
    'Nurburgring': 5148,
    'Kyalami': 4300,
    'Estoril': 4415,
    'A1-Ring': 4318,
    'Donington Park': 4023,
    'Fuji': 4571
    # Aggiungi altri circuiti qui
}.get(circuito, 0) 

# Trovo i giri totali del pilota
giri_totali = session.total_laps

# Trova la scuderia del pilota
scuderia = driver_laps['Team'].iloc[0]

# Ottieni i dati di telemetria per tutti i piloti
all_drivers_telemetry = {}
for drv in session.drivers:
    drv_laps = session.laps.pick_drivers([drv])
    if not drv_laps.empty:
        all_drivers_telemetry[drv] = drv_laps.get_car_data().add_distance()


# Crea un intervallo di timestamp ogni 0.5 secondi
start_time = timedelta(seconds=0)
end_time = weather_data['Time'].max()  # Durata massima della sessione
timestamp_intervals = pd.timedelta_range(start=start_time, end=end_time, freq='500L')  # 500 millisecondi

# Crea un DataFrame vuoto con le nuove colonne
df = pd.DataFrame(columns=[
    'Timestamp', 'Driver', 'Scuderia', 'Track_Temperature', 'Speed',
    'Anno', 'Circuito','Giri_Totali', 'Lunghezza_Circuito', 'Posizione','Giro_in_Corso','Distanza_dal_precedente', 'Distanza_dal_successivo','Metri_Percorsi', 'Percentuale_Completamento_Giro',
    'Tempo_Trascorso_Dal_Giro'
])

# Itera sui timestamp e raccogli i dati
for timestamp in timestamp_intervals:

    # Trova la temperatura della pista più vicina nel tempo
    nearest_weather_idx = (weather_data['Time'] - timestamp).abs().idxmin()
    track_temperature = weather_data.loc[nearest_weather_idx, 'TrackTemp']

    # Trova la velocità del pilota più vicina nel tempo
    nearest_telemetry_idx = (driver_telemetry['Time'] - timestamp).abs().idxmin()
    speed = driver_telemetry.loc[nearest_telemetry_idx, 'Speed']

     # Trova la posizione del pilota più vicina nel tempo
    nearest_position_idx = (driver_laps['Time'] - timestamp).abs().idxmin()
    posizione = driver_laps.loc[nearest_position_idx, 'Position']

     # Trova il giro in corso più vicino nel tempo
    nearest_lap_idx = (driver_laps['LapStartTime'] - timestamp).abs().idxmin()
    giro_in_corso = driver_laps.loc[nearest_lap_idx, 'LapNumber']

 # Trova la distanza dal pilota precedente
    if posizione == 1:
        distanza_dal_precedente = 0  # Se è primo, la distanza è 0
    else:
        # Trova il pilota precedente in classifica
        pilota_precedente = None
        for drv, telemetry in all_drivers_telemetry.items():
            drv_position = session.laps.pick_drivers([drv])['Position'].iloc[0]
            if drv_position == (posizione - 1):
                pilota_precedente = telemetry
                break

        if pilota_precedente is not None:
            # Trova la distanza tra il pilota e quello precedente
            nearest_prec_idx = (pilota_precedente['Time'] - timestamp).abs().idxmin()
            distanza_dal_precedente = driver_telemetry.loc[nearest_telemetry_idx, 'Distance'] - \
                                      pilota_precedente.loc[nearest_prec_idx, 'Distance']
        else:
            distanza_dal_precedente = 0  # Se non ci sono dati, usa 0

    # Trova la distanza dal pilota successivo
    if posizione == len(session.drivers):
        distanza_dal_successivo = 0  # Se è ultimo, la distanza è 0
    else:
        # Trova il pilota successivo in classifica
        pilota_successivo = None
        for drv, telemetry in all_drivers_telemetry.items():
            drv_position = session.laps.pick_drivers([drv])['Position'].iloc[0]
            if drv_position == (posizione + 1):
                pilota_successivo = telemetry
                break

        if pilota_successivo is not None:
            # Trova la distanza tra il pilota e quello successivo
            nearest_succ_idx = (pilota_successivo['Time'] - timestamp).abs().idxmin()
            distanza_dal_successivo = pilota_successivo.loc[nearest_succ_idx, 'Distance'] - \
                                      driver_telemetry.loc[nearest_telemetry_idx, 'Distance']
        else:
            distanza_dal_successivo = 0  # Se non ci sono dati, usa 0
    # Calcola i metri percorsi e la percentuale di completamento del giro
    metri_percorsi = driver_telemetry.loc[nearest_telemetry_idx, 'Distance']
    percentuale_completamento = (metri_percorsi % lunghezza_circuito) / lunghezza_circuito * 100

    # Calcola il tempo trascorso dall'inizio del giro
    lap_start_time = driver_laps.loc[nearest_lap_idx, 'LapStartTime']
    if timestamp >= lap_start_time:
        tempo_trascorso_dal_giro = format_timedelta(timestamp - lap_start_time)
    else:
        tempo_trascorso_dal_giro = "00:00.000" 
    # Aggiungi i dati al DataFrame
    df.loc[len(df)] = [
        timestamp,  # Timestamp
        driver,     # Nome del pilota
        scuderia,   # Scuderia del pilota
        track_temperature,  # Temperatura della pista
        speed,  # Velocità del pilota
        anno,  # Anno
        circuito,  # Nome del circuito
        giri_totali, # Giri totdel pilota
        lunghezza_circuito,  # Lunghezza del circuito in metri
        posizione, # Posizione del pilota
        giro_in_corso, # Giro in corso del pilota
        distanza_dal_precedente,  # Distanza dal pilota precedente
        distanza_dal_successivo,  # Distanza dal pilota successivo
        metri_percorsi,  # Metri percorsi
        percentuale_completamento,  # Percentuale di completamento del giro
        tempo_trascorso_dal_giro  # Tempo nel giro
    ]

# Visualizza il DataFrame risultante
print(df.head(10))

C:\Users\calin\AppData\Local\Temp\ipykernel_24440\2230220934.py:82: FutureWarning: 'L' is deprecated and will be removed in a future version, please use 'ms' instead.
  timestamp_intervals = pd.timedelta_range(start=start_time, end=end_time, freq='500L')  # 500 millisecondi


               Timestamp Driver  Scuderia  Track_Temperature  Speed  Anno  \
0        0 days 00:00:00    HAM  Mercedes               47.4      0  2023   
1 0 days 00:00:00.500000    HAM  Mercedes               47.4      0  2023   
2        0 days 00:00:01    HAM  Mercedes               47.4     12  2023   
3 0 days 00:00:01.500000    HAM  Mercedes               47.4     30  2023   
4        0 days 00:00:02    HAM  Mercedes               47.4     50  2023   
5 0 days 00:00:02.500000    HAM  Mercedes               47.4     67  2023   
6        0 days 00:00:03    HAM  Mercedes               47.4     78  2023   
7 0 days 00:00:03.500000    HAM  Mercedes               47.4    101  2023   
8        0 days 00:00:04    HAM  Mercedes               47.4    128  2023   
9 0 days 00:00:04.500000    HAM  Mercedes               47.4    144  2023   

  Circuito  Giri_Totali  Lunghezza_Circuito  Posizione  Giro_in_Corso  \
0   Monaco           78                3337        5.0            1.0   
1   Mo

In [20]:
print(df.head(50))

                Timestamp Driver  Scuderia  Track_Temperature  Speed  Anno  \
0         0 days 00:00:00    HAM  Mercedes               47.4      0  2023   
1  0 days 00:00:00.500000    HAM  Mercedes               47.4      0  2023   
2         0 days 00:00:01    HAM  Mercedes               47.4     12  2023   
3  0 days 00:00:01.500000    HAM  Mercedes               47.4     30  2023   
4         0 days 00:00:02    HAM  Mercedes               47.4     50  2023   
5  0 days 00:00:02.500000    HAM  Mercedes               47.4     67  2023   
6         0 days 00:00:03    HAM  Mercedes               47.4     78  2023   
7  0 days 00:00:03.500000    HAM  Mercedes               47.4    101  2023   
8         0 days 00:00:04    HAM  Mercedes               47.4    128  2023   
9  0 days 00:00:04.500000    HAM  Mercedes               47.4    144  2023   
10        0 days 00:00:05    HAM  Mercedes               47.4    164  2023   
11 0 days 00:00:05.500000    HAM  Mercedes               47.4   

In [27]:
df.to_csv(f'data {driver} at {circuito} on {anno}.csv', index=False)  # Salva i dati in un file CSV